In [ ]:
# titanic에서 사용된 데이터 전처리 함수 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

# Null처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에서 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[0]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [ ]:
from sklearn.base import BaseEstimator
import numpy as np

# BaseEstimator를 상속받아
# customized 형태의 Estimator를 개발자가 생성할 수 있음
class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음
    def fit(self, X, y=None):
        pass

    # predict() 메서드는 단순히 Sex 피처가 1이면 0, 아니면 1로 예측함
    # 즉, 단순히 성별을 통해 예측함
    def predict(self, X):
        # 데이터의 개수(행 개수) x 1 형태의 ndarray 생성
        pred = np.zeros((X.shape[0],1))

        # 0 ~ 데이터의 개수-1 까지 반복
        for i in range(X.shape[0]):
            # 데이터의 성별 컬럼에서 i번째 행에 대해
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        return pred


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터 로딩, 가공, 학습/테스트 분할
titanic_df = pd.read_csv('/content/train.csv')
y_titanic_df = titanic_df['Survived']

X_titanic_df = titanic_df.drop('Survived',axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                    test_size=0.2, random_state=0)

myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)

# 성별만 가지고 단순히 예측했으나 0.7877이라는 정확도가 나옴
# 즉, 정확도를 평가 지표로 사용할 때는 신중해야함
print("Dummy Classifier의 정확도 : {0:.4f}".format(accuracy_score(y_test,mypredictions)))

Dummy Classifier의 정확도 : 0.7877


In [ ]:
# MNIST 데이터를 활용한 Accuracy 문제점 분석
# 7인 것만 true, 나머지는 false로 하여 이진 분류 문제로 변형
# 불균형한 데이터 세트 (하나의 특성이 10% 90%로 나뉨)에 대해서는 Accuracy는 평가지표로 적합하지 않음

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self,X,y):
        pass
    
    # 입력 값으로 들어오는 X 데이터 세트의 크기만큼 무조건 모두 0값으로 만들고 반환
    # 즉, 아무런 학습 없이 단지 데이터 개수만큼 0값 반환
    def pred(self, X):
        return np.zeros((len(X),1), dtype=bool)

digits = load_digits()

# 7이면 true이고 이를 astype(int)로 1로 변환
# 7이 아니면 False이고 0으로 변환
y = (digits.target ==7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y,
                                                    test_size=0.2, random_state=0)

print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
# ndarray -> series 변환 후, 값 분포 확인
print(pd.Series(y_test).value_counts())

# Dummy Classfier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.pred(X_test)

print("모든 예측을 0으로 하여도 정확도 : {0:.4f}".format(accuracy_score(y_test,fakepred)))

레이블 테스트 세트 크기 : (360,)
테스트 세트 레이블 0과 1의 분포도
0    321
1     39
dtype: int64
모든 예측을 0으로 하여도 정확도 : 0.8917
